In [1]:
import torch
import os
import numpy as np
import pickle
import utils
import model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

c:\Users\leonp\anaconda3\envs\ATCS_6\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("First load/build the vocab and embeddings")
dataset_path = "../data/"

if os.path.isfile(dataset_path + "embeddings.txt"):
    print("Found required files")
    vectors = np.loadtxt(dataset_path + "embeddings.txt")
    vectors = torch.Tensor(vectors)

    with open(dataset_path + "vocab.pickle", "rb") as f:
        vocab = pickle.load(f)
else:
    vocab, vectors = utils.create_vocab(dataset_path)
    vectors = torch.Tensor(vectors)

First load/build the vocab and embeddings
Found required files


In [20]:
# decide which model you want to use
# base = model.Baseline(vectors).to(device)
# net = model.NeuralNet(base, 300).to(device)

# lstm = model.LSTM(vectors, 2048).to(device)
# net = model.NeuralNet(lstm, 2048).to(device)

# bi_lstm = model.BiLSTM(vectors, 4096, False).to(device)
# net = model.NeuralNet(bi_lstm, 4096).to(device)

bi_lstm = model.BiLSTM(vectors, 4096, True).to(device)
net = model.NeuralNet(bi_lstm, 4096).to(device)

In [21]:
def pred(vocab, net, device, premise, hypothesis):
    premise, hypothesis, _ = utils.process_sentence(premise, hypothesis, [1], vocab, device)
    premise = premise
    hypothesis = hypothesis

    preds = net(premise, hypothesis)

    preds = torch.argmax(preds, dim=-1)

    for item in preds:
        if int(item.item()) == 0:
            return "Entailment"
        if int(item.item()) == 1:
            return "Neutral"

        return "Contradiction"

In [22]:
premise = ["Two men sitting in the sun"]
hypothesis = ["Nobody is sitting in the shade"]
pred(vocab, net, device, premise * 2, hypothesis * 2)

'Contradiction'

In [23]:
premise = ["A man is walking a dog"]
hypothesis = ["No cat is outside"]
pred(vocab, net, device, premise * 2, hypothesis * 2)

'Contradiction'